In [333]:
import math
import random

# Constants
f_S_TYP = 100000 # [Hz] 100 kHz
f_S_MIN = f_S_TYP * 0.9
f_S_MAX = f_S_TYP * 1.05

f_D_TYP = 1000000 # [Hz] 1 MHz
f_D_MIN = f_D_TYP * 0.9
f_D_MAX = f_D_TYP * 1.1

In [334]:
def print_capacitance(capacitance):
    if capacitance < 1e-9:
        return str(round(capacitance*1e12, 2)) + " pF"
    elif capacitance < 1e-6:
        return str(round(capacitance*1e9, 2)) + " nF"
    else:
        return str(round(capacitance*1e6, 2)) + " uF"

def pretty_capacitors(capacitors):
    strings = []
    for capacitor in capacitors:
        if capacitor != 0:
            strings.append(print_capacitance(capacitor))
    return " + ".join(strings) + " = " + print_capacitance(sum(capacitors))

def generate_capacitors():
    standard = [10, 12, 15, 18, 22, 27, 33, 39, 47, 56, 68, 82]
    magnitudes = [1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12] # uF, nF, pF
    capacitors = [0]
    for mag in magnitudes:
        for std in standard:
            capacitors.append(std*mag)
    return capacitors

def generate_capacitor_combos():
    capacitors = generate_capacitors()
    combos = []
    for i in range(len(capacitors)):
        for j in range(len(capacitors)):
            sum_capacitance = sum([capacitors[i], capacitors[j]])
            if len(combos) == 0 or sum_capacitance != combos[-1]:
                combos.append([capacitors[i], capacitors[j]])
    combos.sort(key=sum)
    return combos

capacitor_combos_3 = generate_capacitor_combos()
# export as csv
with open('capacitor_combos.csv', 'w') as f:
    f.write(",".join(["C", "C1", "C2"]) + "\n")
    for combo in capacitor_combos_3:
        f.write(str(sum(combo)) + "," + ",".join([str(x) for x in combo]) + "\n")
print("Number of Capacitor Combinations:", len(capacitor_combos_3))

def select_capacitors(target_capacitance):
    best_capacitors = capacitor_combos_3[0]
    best_capacitance = sum(best_capacitors)

    # bottom up search
    for i in range(len(capacitor_combos_3)):
        capacitance = sum(capacitor_combos_3[i])
        if capacitance > target_capacitance:
            continue
        if target_capacitance - capacitance < target_capacitance - best_capacitance:
            best_capacitors = capacitor_combos_3[i]
            best_capacitance = capacitance

    best_capacitors.sort()
    best_capacitors.reverse()
    return best_capacitors

target_capacitance = 1372*1e-9
print("Target Capacitance:", print_capacitance(target_capacitance))
print("Capacitor Combination:", pretty_capacitors(select_capacitors(target_capacitance)))


Number of Capacitor Combinations: 14641
Target Capacitance: 1.37 uF
Capacitor Combination: 680.0 nF + 680.0 nF = 1.36 uF


In [335]:
# Coil
uL_iS =  16 # [uH]
uL_S =  11 # [uH]
R = 0.191 # [Ohm]

In [336]:
# C * 1e12 = pF
# C * 1e9 = nF
# C * 1e6 = uF

L_iS = uL_iS * 1e-6 # [H]
L_S = uL_S * 1e-6 # [H]

def caclulate_C_1(f_S, L_iS):
    C_1 = 1 / ( pow(2 * math.pi * f_S, 2) * L_iS )
    return C_1

C_1_TYP = caclulate_C_1(f_S_TYP, L_iS)
print("C_1 =", print_capacitance(C_1_TYP))
print(pretty_capacitors(select_capacitors(C_1_TYP)))

def caclulate_C_2(f_D, L_S, C_1):
    C_2 = 1 / ( pow(2 * math.pi * f_D, 2) * L_S - (1/C_1) )
    return C_2

C_2_TYP = caclulate_C_2(f_D_TYP, L_S, C_1_TYP)
print("C_2 =", print_capacitance(C_2_TYP))
print(pretty_capacitors(select_capacitors(C_2_TYP)))


Q = (2 * math.pi * f_D_TYP * L_S) / R
print("Q =", round(Q), "Ω")

C_1 = 158.31 nF
150.0 nF + 8.2 nF = 158.2 nF
C_2 = 2.34 nF
2.2 nF + 120.0 pF = 2.32 nF
Q = 362 Ω
